In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy import linalg
from datetime import datetime, timedelta
import scipy.optimize as optimization
from scipy.integrate import odeint
from scipy.stats import poisson
from ddeint import ddeint

In [2]:
# Please see "data-tailor-COVID19-Italy" on how to generate the italydata.csv file.

data=pd.read_csv("./italydata.csv",low_memory=False)
dataframe=pd.DataFrame(data)

In [3]:
# Model parameters for the first outbreak wave.

ODEtime1=np.linspace(0,150,151)
S0=60.36*10**6
L0=3223
I0=1500
PC0=26
PM0=103
R0=1
rho=1/5.2
kappa=1/6
p1=0.0378
gammapc=0.1
gammapm=0.125
mupc1=0.2050
mupm=0

def betafunc1(t):
    return 0.12*5.2*math.exp(-0.038*t)/S0

In [4]:
# Transmission model for the first outbreak wave.

def ODEmodel1(x,tt):
    SS,LL,II,PC,PM,DD,CC,RR=x
    t=np.floor(tt)
    return np.array([-betafunc1(t)*SS*II, betafunc1(t)*SS*II-rho*LL, rho*LL-kappa*II,
                    p1*PM-(mupc1+gammapc)*PC, kappa*II-(gammapm+p1)*PM, 
                    mupc1*PC, kappa*II, gammapm*PM])

ODEsol1=odeint(ODEmodel1,(S0,L0,I0,PC0,PM0,2,132,1),ODEtime1)

In [64]:
# Model parameters for the second outbreak wave.

ODEtime2=np.linspace(210,410,201)
S1=60.061844*10**6
L1=7277.8
I1=1000
PC1=222
PM1=43876
R0=218351
rho=1/5.2
kappa=1/6
p2=0.0038
gammapc=0.1
gammapm=0.125
mupc2=0.1457
mupm=0

def betafunc2(t):
    return 0.12*5.3688*math.exp(-0.025*(t-210))/S1

In [26]:
# Transmission model for the second outbreak wave.

def ODEmodel2(x,tt):
    SS,LL,II,PC,PM,DD,CC=x
    t=np.floor(tt)
    return np.array([-betafunc035(t)*SS*II, betafunc035(t)*SS*II-rho*LL, rho*LL-kappa*II,
                    p2*PM-(mupc2+gammapc)*PC, kappa*II-(gammapm+p2)*PM, 
                    mupc2*PC, kappa*II])

ODEsol2=odeint(ODEmodel2,(S1,L1,I1,PC1,PM1,35707,298156),ODEtime2)

In [ ]:
# Generate newly identified case inflow rate for the blood transfusion model.

inflowlist=[]
for i in range(0,277):
    inflowlist.append(dataframe['daily_case'][i])
for i in range(277,410):
    inflowlist.append(ODEsol2[i-210,2]*kappa)